In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import matplotlib as mpl
from scipy.optimize import curve_fit
from scipy.stats import binned_statistic

In [ ]:
import vice

from surp import subgiants
from surp import gce_math as gcem
from surp.gce_math import MH_to_Z, Z_to_MH, Z_SUN
import surp

import arya

In [ ]:
arya.style.set_size((4, 2*np.sqrt(2)))

In [ ]:
surp.set_yields()

In [ ]:
# all_abundances = pd.read_csv("data_analysis/all_data.csv")
# stars = pd.read_csv("../data/amarsi_19.tsv", delimiter="\t *", comment="#", skiprows=[62, 63], engine="python")
# stars["C_MG"] =  stars["[C/H]3N"] - stars["[O/H]3N"]
# stars["MG_H"] =  stars["[O/H]3N"]
# stars["C_MG_ERR"] = stars["e_[C/H]3N"] + stars["e_[O/H]3N"]

# df_stars = stars[["C_MG", "MG_H", "C_MG_ERR"]]

# all_abundances["C_MG"] = all_abundances["[c/o]"]
# all_abundances["MG_H"] = all_abundances["[o/h]"]
# all_abundances["C_MG_ERR"] = all_abundances["[c/o]_err"]

# stars_all = all_abundances[["C_MG", "MG_H", "C_MG_ERR"]][(~np.isnan(all_abundances.C_MG)) & (all_abundances.type == "star")]
# RL_all = all_abundances[["C_MG", "MG_H", "C_MG_ERR"]][(~np.isnan(all_abundances.C_MG)) & (all_abundances.type == "RL")]
# DLA_all = all_abundances[["C_MG", "MG_H", "C_MG_ERR"]][(~np.isnan(all_abundances.C_MG)) & (all_abundances.type == "DLA")]
# dwarf_all = all_abundances[["C_MG", "MG_H", "C_MG_ERR"]][(~np.isnan(all_abundances.C_MG)) & (all_abundances.type == "dwarf")]

In [ ]:
# stars_x_obs = stars.MG_H
# stars_y_obs = stars.C_MG
# all_x_obs = all_abundances.MG_H
# all_y_obs = all_abundances.C_MG

In [ ]:

filt = ~subgiants.high_alpha.values
df = subgiants[filt]

bins = np.quantile(df.MG_H, np.linspace(0, 1, 20))

counts = binned_statistic(df.MG_H, df.C_MG, bins=bins, statistic='count')[0]

ym = binned_statistic(df.MG_H, df.C_MG, bins=bins, statistic='median')[0]
yl = binned_statistic(df.MG_H, df.C_MG, bins=bins, statistic=lambda x: np.percentile(x, 16))[0]
yh = binned_statistic(df.MG_H, df.C_MG, bins=bins, statistic=lambda x: np.percentile(x, 84))[0]
ym_err = binned_statistic(df.MG_H, df.C_MG_ERR, bins=bins, statistic='mean')[0]


xm = binned_statistic(df.MG_H, df.MG_H, bins=bins, statistic='median')[0]
xl = binned_statistic(df.MG_H, df.MG_H, bins=bins, statistic=lambda x: np.percentile(x, 16))[0]
xh = binned_statistic(df.MG_H, df.MG_H, bins=bins, statistic=lambda x: np.percentile(x, 84))[0]
xm_err = binned_statistic(df.MG_H, df.MG_H_ERR, bins=bins, statistic='mean')[0]

In [ ]:
yym = gcem.brak_to_abund_ratio(ym, "c", "mg")
yyl = gcem.brak_to_abund_ratio(yl - ym_err, "c", "mg")
yyh = gcem.brak_to_abund_ratio(yh + ym_err, "c", "mg")
yym_err = (yyh - yyl) / 2 / np.sqrt(counts)

x_err = (xh - xl)/ 2 / np.sqrt(counts)

In [ ]:
obs = (xm, yym, x_err, yym_err)

In [ ]:
def plot_obs():
    plt.errorbar(xm, yym, yerr=yym_err, xerr=x_err,  fmt=".", capsize=0)

In [ ]:
plt.hist2d(df.MG_H, gcem.brak_to_abund_ratio(df.C_MG, "c", "mg"), 50, range=([-0.7, 0.7], [3, 6]), cmap="Greys")
plot_obs()


In [ ]:
params = surp.YieldParams.from_file("../models/fruity/f_0/yield_params.toml")
surp.set_yields(params)
y_c_lin = vice.yields.ccsne.settings["c"]

surp.set_yields(surp.YieldParams.from_file("../models/fiducial/yield_params.toml"))
y_c_quad = vice.yields.ccsne.settings["c"]

In [ ]:
params

In [ ]:
ymg = vice.yields.ccsne.settings["mg"]

In [ ]:
alpha = params.alpha_c_agb

In [ ]:
y_c_lin = np.vectorize(y_c_lin)
y_c_quad = np.vectorize(y_c_quad)
def y_c_agb(Z):
    return y_agb_0 + zeta_agb * gcem.Z_to_MH(Z)

In [ ]:
plot_obs()
x = np.linspace(-0.45, 0.45, 1000)
Z = gcem.MH_to_Z(x)
# plt.plot(x, (y_c_lin(Z) + y_c_agb(Z)) / ymg, label="linear fit")
# plt.plot(x,(y_c_quad(Z) + y_c_agb(Z))/ymg, ls="--", label="quadratic fit")

plt.legend()
plt.xlabel("[Mg/H]")
plt.ylabel(r"$y_{\rm C} / y_{\rm Mg}$")
plt.savefig("figures/equilibrium_yield.pdf")

# LS Fits

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def loglin(x, a, b):
    return a + b*x

In [ ]:
popt, covt = curve_fit(loglin, xm, yym, sigma=yym_err)

In [ ]:
x_pred = np.linspace(-0.6, 0.6, 1000)
y_pred = loglin(x_pred, *popt)
plt.plot(x_pred, y_pred)
plot_obs()

In [ ]:
def lin(x, a, b):
    Z = gcem.MH_to_Z(x)
    return a + b*(Z - Z_SUN)

In [ ]:
popt, covt = curve_fit(lin, xm, yym, sigma=yym_err)

In [ ]:
popt

In [ ]:
x_pred = np.linspace(-0.6, 0.5, 1000)
y_pred = lin(x_pred, *popt)
plt.plot(x_pred, y_pred)
plot_obs()

In [ ]:
def logquad(x, a, b, c):
    return a + b*x + c*x**2

In [ ]:
popt, covt = curve_fit(logquad, xm, yym, sigma=yym_err)

In [ ]:
popt

In [ ]:
x_pred = np.linspace(-0.6, 0.5, 1000)
y_pred = logquad(x_pred, *popt)
plt.plot(x_pred, y_pred)
plot_obs()

In [ ]:
def logexp(x, a, b, c):
    return a + b*np.exp(x*c)

In [ ]:
popt, covt = curve_fit(logexp, xm, yym, sigma=yym_err)

In [ ]:
popt, np.sqrt(np.diag(covt))

In [ ]:
x_pred = np.linspace(-0.6, 0.5, 1000)
y_pred = logexp(x_pred, *popt)
plt.plot(x_pred, y_pred)
plot_obs()
plt.xlabel("[M/H]")
plt.ylabel(r"$y_{\rm C}/y_{\rm Mg}$")

In [ ]:
def logloglin(x, a, b, c):
    return 10**(a + b*x + c*x**2)

In [ ]:
popt, covt = curve_fit(logloglin, xm, yym, sigma=yym_err)
print(popt)
x_pred = np.linspace(-0.6, 0.5, 1000)
y_pred = logloglin(x_pred, *popt)
plt.plot(x_pred, y_pred)
plot_obs()
plt.yscale("log")

In [ ]:
def piecewise(x, a, b, a0):
    return np.maximum(a + b*x, a0)

In [ ]:
y = surp.yield_models.BiLogLin_CC(y0=4, y1=4, zeta=2.5)

In [ ]:
popt, covt = curve_fit(piecewise, xm, yym, sigma=yym_err, p0=(4.5, 0, 4.2))
print(popt)
x_pred = np.linspace(-0.6, 0.5, 1000)
y_pred = piecewise(x_pred, *popt)
plt.plot(x_pred, y_pred)
plt.plot(x_pred, [y(Z) for Z in gcem.MH_to_Z(x_pred)])
plot_obs()
plt.yscale("log")

# MCMC fits

In [ ]:
N =3_000

In [ ]:
import mcmc_utils

In [ ]:
def plot_res(model, samples):
    theta_m = np.median(samples, axis=0)
    y_pred = model(xm, theta_m)

    #plt.scatter(x_obs, y_obs - y_pred, s=2, c=w, ec="none", cmap="Greys", alpha=0.4)
    plt.scatter(xm, yym - y_pred, )
    plt.ylim(-0.3, 0.3)
    plt.xlim(-0.5)
    plt.axhline(0, ls="--")

## Linear Model

In [ ]:
model = mcmc_utils.Linear(beta0=[4, 0.5], beta1=[0, 1], log_sigma=[-15, 0.5])

In [ ]:
mcmc_utils.plot_prior(model, N=1_000)

In [ ]:
samples = mcmc_utils.run_mcmc(model, obs,N=N )

In [ ]:
mcmc_utils.plot_posterior(model, obs, samples, N_lines=500)

# Exponential model

In [ ]:
model  = mcmc_utils.Exponential(beta0=[4, 0.5], beta1=[0.0, 0.4], log_sigma=[-120, 1], alpha=[0, 4])

In [ ]:
mcmc_utils.plot_prior(model, xlims=(-0.5, 0.5))
plt.ylim(3, 5)

In [ ]:
samples = mcmc_utils.run_mcmc(model, obs, N=N)

In [ ]:
mcmc_utils.plot_posterior(model, obs, samples)

## Quadratic model

In [ ]:
model  = mcmc_utils.Quadratic(beta0=[4, 0.5], beta1=[0, 1], log_sigma=[-15, 1])

In [ ]:
mcmc_utils.plot_prior(model)

In [ ]:
samples = mcmc_utils.run_mcmc(model, obs, N=N)

In [ ]:
mcmc_utils.plot_posterior(model, obs, samples)